In [ ]:
# % pip install groundingdino-py segment-geospatial

# Testing out point prompting for log pile segmentation

This example is adapted from https://samgeo.gishub.org/examples/input_prompts/


In [ ]:
from samgeo import SamGeo, tms_to_geotiff

In [ ]:
bbox = [-106.379, 40.018, -106.37, 40.0225]

In [ ]:
image = "satellite.tif"
tms_to_geotiff(output=image, bbox=bbox, zoom=19, source="Satellite", overwrite=True)

In [ ]:
sam = SamGeo(
    model_type="vit_h",
    automatic=False,
    sam_kwargs=None,
)

In [ ]:
sam.set_image(image)

In [ ]:
log_coords = [
    [-106.3748701, 40.0208968],
    [-106.3746750, 40.0211880],
    [-106.3734266, 40.0212524],
    [-106.3753353, 40.0215654],
    [-106.37515987, 40.02019197],
    [-106.3747749, 40.0202933],
    [-106.3741671, 40.0209093],
    [-106.3731381, 40.0205163], 
]

background_coords = [
    [-106.3750153, 40.0211513],
    [-106.3759026, 40.0210525],
    [-106.3757408, 40.0203401],
    [-106.3737822, 40.0203241],
]

n_log_pts = len(log_coords)
n_background_pts = len(background_coords)

point_coords = log_coords + background_coords

point_labels = [1] * n_log_pts + [0] * n_background_pts

In [ ]:
sam.predict(point_coords, point_labels=point_labels, point_crs="EPSG:4326", output="mask_w_bg.tif")

Finally, save the labeled points as a geopackage to visualize in a GIS. 

In [ ]:
import geopandas as gpd

gdf = gpd.GeoDataFrame(
    {"label": point_labels}, 
    geometry=gpd.points_from_xy(
        x=[p[0] for p in point_coords], 
        y=[p[1] for p in point_coords],
    ), 
    crs=4326
)

gdf.to_file("labels.gpkg")